In [2]:
import pandas as pd
import numpy as np
import random
import pickle

In [56]:

pipeline = pickle.load(open('../models/pipeline_rf.pkl', 'rb'))
data = pd.read_csv('../data/processed/test.csv')

In [ ]:
m

In [57]:
pipeline.predict(data)

ValueError: X has 34 features, but ColumnTransformer is expecting 10 features as input.

In [5]:
data.head()

,dTime,dTimeUTC,aTime,aTimeUTC,airlines,fly_duration,flyFrom,cityFrom,cityCodeFrom,flyTo,...,session,orig-dest,airline,days_until_dep,hops,direct,competition,id,hist_prices,waiting_days
0,15:00,2021-03-17 14:00:00,16:25,2021-03-17 15:25:00,UX,1.416667,MAD,Madrid,MAD,BCN,...,afternoon,MAD-BCN,UX,45,0,True,17,67731,"[34, 34, 34, 46, 46, 46, 46, 46, 38, 38, 38, 3...",0
1,15:00,2021-03-17 14:00:00,16:25,2021-03-17 15:25:00,UX,1.416667,MAD,Madrid,MAD,BCN,...,afternoon,MAD-BCN,UX,44,0,True,17,67731,"[34, 34, 34, 46, 46, 46, 46, 46, 38, 38, 38, 3...",0
2,15:00,2021-03-17 14:00:00,16:25,2021-03-17 15:25:00,UX,1.416667,MAD,Madrid,MAD,BCN,...,afternoon,MAD-BCN,UX,43,0,True,17,67731,"[34, 34, 34, 46, 46, 46, 46, 46, 38, 38, 38, 3...",0
3,15:00,2021-03-17 14:00:00,16:25,2021-03-17 15:25:00,UX,1.416667,MAD,Madrid,MAD,BCN,...,afternoon,MAD-BCN,UX,42,0,True,17,67731,"[34, 34, 34, 46, 46, 46, 46, 46, 38, 38, 38, 3...",5
4,15:00,2021-03-17 14:00:00,16:25,2021-03-17 15:25:00,UX,1.416667,MAD,Madrid,MAD,BCN,...,afternoon,MAD-BCN,UX,41,0,True,17,67731,"[34, 34, 34, 46, 46, 46, 46, 46, 38, 38, 38, 3...",4


In [64]:
def get_waiting_price(row):
    if row['predicted'] != 0:
        #idx = row['predicted'] - 1
        list_prices = row['hist_prices'].strip('][').split(', ')
        idx = len(list_prices)-row['days_until_dep']+row['predicted']
        waiting_price = float(list_prices[idx])
        if np.isnan(waiting_price):
            waiting_price = row['price']
    else:
        waiting_price = row['price']
        
    return waiting_price

In [63]:
def get_best_waiting_price(row):
    if row['waiting_days'] != 0:
        #idx = row['waiting_days'] - 1
        list_prices = row['hist_prices'].strip('][').split(', ')
        idx = len(list_prices)-row['days_until_dep']+row['waiting_days']
        waiting_price = float(list_prices[idx])
        if np.isnan(waiting_price):
            waiting_price = row['price']
    else:
        waiting_price = row['price']
        
    return waiting_price

In [58]:
class Simulator():
    def __init__(self, n, flights, pipeline):
        self.n = n
        self.flights = flights
        self.pipeline = pipeline
        
    def generate_travellers(self):
        """
        routes = self.flights['orig-dest'].unique()
        departures = self.flights['dDate'].unique()
        
        #requests = self.flights['days_until_dep'].unique()
        requests = np.arange(7, 46)
        travellers_routes = random.choices(routes, k=self.n)
        travellers_dep_date = random.choices(departures, k=self.n)
        travellers_req_date = random.choices(requests, k=self.n)
        id_traveler = np.arange(0, self.n)
        self.travellers = pd.DataFrame({'id_traveler': id_traveler,
                                        'route': travellers_routes,
                                        'departure': travellers_dep_date,
                                        'request':travellers_req_date})
        """
        self.travellers = self.flights[['orig-dest','dDate', 'days_until_dep']].sample(self.n)
        self.travellers['id_traveler'] = np.arange(0, self.n)
        
    def get_cheapest_flights(self):
        # select cheapest flight for each traveler
        #merged = pd.merge(self.travellers, self.flights, left_on=['departure', 'route', 'request'], right_on=['dDate', 'orig-dest', 'days_until_dep'])
        merged = pd.merge(self.travellers, self.flights, on=['orig-dest','dDate', 'days_until_dep'])
        #group_cols = list(self.travellers.columns)
        cheapest_indexes = merged.groupby('id_traveler')['price'].idxmin()
        self.cheapest_flights = merged.loc[cheapest_indexes]

    def prepare_data(self, data):
        num_attribs = ['days_until_dep', 'fly_duration', 'day_of_month', 'log_price', 'hops', 'competition']
        cat_attribs = ['flyFrom', 'flyTo', 'day_of_week', 'session']
    
        #data_prepared = self.pipeline.transform(data[num_attribs+cat_attribs])
        return data[num_attribs+cat_attribs]
    
    def make_predictions(self):
        data_prepared = self.prepare_data(self.cheapest_flights)
        
        predicted = self.pipeline.predict(data_prepared).round()
        self.cheapest_flights['predicted'] = predicted.round().astype(int)
        
    def compute_savings(self):
        df = self.cheapest_flights
        df['waiting_price'] = df.apply(get_waiting_price, axis=1)
        df['best_waiting_price'] = df.apply(get_best_waiting_price, axis=1)
        df['savings'] = df['price'] - df['waiting_price']
        
    def visualize_results(self):
        df = self.cheapest_flights[['price', 'predicted', 'waiting_price', 'savings']]
        
        current_sum = df['price'].sum()
        current_mean = df['price'].mean()
        
        model_sum = df['waiting_price'].sum()
        model_mean = df['waiting_price'].mean()
        
        savings_sum = df['savings'].sum()
        savings_mean = df['savings'].mean()
        
        
        
        percent = str(round(savings_sum/current_sum*100, 2)) + '%'
        
        results = {'Current': [current_sum, current_mean],
                   'Model': [model_sum, model_mean],
                   'Savings':[savings_sum, savings_mean],
                   'Percentage': [percent, percent]}
        
        df = pd.DataFrame(results, index=['Total (€)', 'Per traveler (€)'])
        df = df.round(2)
        print(df.to_string())
        
    def visualize_results_by_route(self):
        
        df = sim.cheapest_flights[['orig-dest', 'price', 'waiting_price', 'savings', 'best_waiting_price']]
        by_route = df.groupby('orig-dest')[['price', 'waiting_price', 'savings', 'best_waiting_price']].agg(['sum', 'mean'])
        
        
        by_route['percentage'] = (by_route['savings']['sum']/by_route['price']['sum']*100).round(2).astype(str) + '%'
        by_route['max_percentage'] = ((by_route['price']['sum']-by_route['best_waiting_price']['sum'])/by_route['price']['sum']*100).round(2).astype(str) + '%'
        print(by_route.to_string())
        
        print('\nOnly waiting predictions:\n')
        wait = sim.cheapest_flights['waiting_days'] != 0
        only_wait = sim.cheapest_flights[wait]
        df = only_wait[['orig-dest', 'price', 'waiting_price', 'savings', 'best_waiting_price']]
        by_route = df.groupby('orig-dest')[['price', 'waiting_price', 'savings', 'best_waiting_price']].agg(['sum', 'mean'])
        by_route['percentage'] = (by_route['savings']['sum']/by_route['price']['sum']*100).round(2).astype(str) + '%'
        by_route['max_percentage'] = ((by_route['price']['sum']-by_route['best_waiting_price']['sum'])/by_route['price']['sum']*100).round(2).astype(str) + '%'
        print(by_route.to_string())
        
    def run(self):
        self.generate_travellers()
        self.get_cheapest_flights()
        self.make_predictions()
        self.compute_savings()
        self.visualize_results_by_route()
    

In [66]:
sim = Simulator(1000, data, pipeline)

In [67]:
sim.run()

           price             waiting_price             savings            best_waiting_price             percentage max_percentage
             sum        mean           sum        mean     sum       mean                sum        mean                          
orig-dest                                                                                                                         
BCN-AMS    14471  113.944882       14748.0  116.125984  -277.0  -2.181102            13653.0  107.503937     -1.91%          5.65%
BCN-FCO    14146  141.460000       14813.0  148.130000  -667.0  -6.670000            13120.0  131.200000     -4.72%          7.25%
BCN-LGW     5928  116.235294        6041.0  118.450980  -113.0  -2.215686             5640.0  110.588235     -1.91%          4.86%
BCN-PMI     7324   51.943262        7479.0   53.042553  -155.0  -1.099291             7120.0   50.496454     -2.12%          2.79%
MAD-BCN    12729   51.743902       12756.0   51.853659   -27.0  -0.109756          

In [22]:
sim.flights[['departure']]

,id_traveler,route_x,departure,request,dTime,dTimeUTC,aTime,aTimeUTC,airlines,fly_duration,...,session,orig-dest,airline,days_until_dep,hops,direct,competition,id,hist_prices,waiting_days
0,2,BCN-LGW,2021-03-18,19,07:00,2021-03-18 06:00:00,08:20,2021-03-18 08:20:00,VY,2.333333,...,morning,BCN-LGW,VY,19,0,True,20,69873,"[22, 58, 59, 104, 104, 104, 104, 92, 92, 104, ...",3
1,3,MAD-MEX,2021-03-25,22,06:40,2021-03-25 05:40:00,17:30,2021-03-25 23:30:00,AF,17.833333,...,morning,MAD-MEX,AF,22,1,False,33,90846,"[325, 325, 325, 325, 325, 355, 355, 355, 355, ...",15
2,3,MAD-MEX,2021-03-25,22,16:50,2021-03-25 15:50:00,19:25,2021-03-27 01:25:00,KL,33.583333,...,afternoon,MAD-MEX,KL,22,1,False,33,91414,"[420, 420, 426, 426, 426, 358, 358, 358, 358, ...",14
3,4,MAD-EZE,2021-03-28,7,11:10,2021-03-28 09:10:00,06:40,2021-03-29 09:40:00,"LH, TP",24.500000,...,morning,MAD-EZE,LH,7,2,False,25,103285,"[3093, 3046, 3089, 3042, 3114, 3052, 3049]",3
4,4,MAD-EZE,2021-03-28,7,11:10,2021-03-28 09:10:00,06:40,2021-03-29 09:40:00,"LH, TP",24.500000,...,morning,MAD-EZE,LH,7,2,False,25,103285,"[3093, 3046, 3089, 3042, 3114, 3052, 3049]",3
5,7,MAD-MEX,2021-03-25,30,06:40,2021-03-25 05:40:00,17:30,2021-03-25 23:30:00,AF,17.833333,...,morning,MAD-MEX,AF,30,0,True,33,90846,"[325, 325, 325, 325, 325, 355, 355, 355, 355, ...",6
6,7,MAD-MEX,2021-03-25,30,16:50,2021-03-25 15:50:00,19:25,2021-03-27 01:25:00,KL,33.583333,...,afternoon,MAD-MEX,KL,30,0,True,33,91414,"[420, 420, 426, 426, 426, 358, 358, 358, 358, ...",6
7,8,BCN-AMS,2021-03-24,18,20:35,2021-03-24 19:35:00,09:55,2021-03-25 08:55:00,UX,13.333333,...,evening,BCN-AMS,UX,18,1,False,21,85697,"[61, 61, 102, 125, 125, 125, 125, 125, 81, 81,...",0
8,8,BCN-AMS,2021-03-24,18,12:30,2021-03-24 11:30:00,15:00,2021-03-24 14:00:00,HV,2.500000,...,afternoon,BCN-AMS,HV,18,0,True,21,85622,"[134, 134, 87, 87, 87, 87, 87, 87, 87, 87, 87,...",0


In [14]:
cheapest_indexes = merged.groupby('id_traveler')['price'].idxmin()

In [21]:
cheapest_indexes

id_traveler
2    0
3    1
4    3
7    6
8    8
Name: price, dtype: int64

In [ ]:
sim.generate_travellers()
sim.get_cheapest_flights()

In [27]:
sim = Simulator(5000, data, model, pipeline)
sim.run()

            price             waiting_price              savings            best_waiting_price             percentage max_percentage
              sum        mean           sum        mean      sum       mean                sum        mean                          
orig-dest                                                                                                                           
BCN-AMS     41113   91.975391       41402.0   92.621924   -289.0  -0.646532            38289.0   85.657718      -0.7%          6.87%
BCN-FCO     52238  122.337237       54098.0  126.693208  -1860.0  -4.355972            48949.0  114.634660     -3.56%           6.3%
BCN-LGW     18797   78.978992       19361.0   81.348739   -564.0  -2.369748            18601.0   78.155462      -3.0%          1.04%
BCN-PMI     12710   31.228501       12830.0   31.523342   -120.0  -0.294840            11808.0   29.012285     -0.94%           7.1%
MAD-BCN     26250   51.269531       26744.0   52.234375   -494.0  -0.

In [19]:
df = sim.cheapest_flights

In [21]:
((df['price'] - df['best_waiting_price']) > 0).sum()/sim.n*100

29.459999999999997